In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from scipy import stats
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv

In [2]:
rlist = []
records = os.path.normpath('mit-bih-dataframes/subject_list.csv')
with open(records) as rfile:
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [3]:
rr_ints = []
for x in tqdm(rlist):
    rr_ints.append(pd.read_csv(os.path.normpath('mit-bih-extracted/'+x+'_rr_rhythms.csv'), index_col=0))

rr_ints[22].tail()

100%|███████████████████████████████████████████| 23/23 [00:01<00:00, 22.18it/s]


,rr_int,rr_int_seconds,rhythms,rmean,rmean_seconds,drmean
59470,173,0.692,A,99.326854,0.397307,24.557715
59471,157,0.628,A,113.745140,0.454981,14.418287
59472,68,0.272,A,102.308855,0.409235,-11.436285
59473,69,0.276,A,93.981641,0.375927,-8.327214
59474,53,0.212,A,83.736231,0.334945,-10.245410


In [5]:
def subset_subject(subjdf, interval_length = 25):
    if (len(subjdf) % interval_length != 0):
        slices = len(subjdf) // interval_length + 1
    else:
        slices = int(len(subjdf) / interval_length)

    subsetdf = []
    for x in tqdm(range(slices)):
        subsetdf.append(subjdf.iloc[interval_length*x:interval_length*x+interval_length])
    return subsetdf

In [16]:
for idx, subject in enumerate(rr_ints):
    subsets = subset_subject(subject)
    idx_list = list(range(len(subsets)))

    data = {
        "subsetID": [str(rlist[idx])+"-"+str(idx_list[x])+".csv" for x in idx_list],
        "rhythmLabel": [subsets[x]['rhythms'].mode().iloc[0] for x in idx_list]
    }
    
    subset_list = pd.DataFrame(data)
    subset_list.to_csv(os.path.normpath('mit-bih-subsets/'+rlist[idx]+"_subset_list.csv"))
    print(subset_list.head())
    
    os.makedirs('mit-bih-subsets/'+str(rlist[idx]), exist_ok=True)

    for x, subset in enumerate(subsets):
        subset.to_csv(os.path.normpath('mit-bih-subsets/'+str(rlist[idx])+'/'+str(rlist[idx])+"-"+str(idx_list[x])+".csv"))

100%|████████████████████████████████████| 1713/1713 [00:00<00:00, 25406.36it/s]


      subsetID rhythmLabel
0  04015-0.csv           N
1  04015-1.csv           N
2  04015-2.csv           N
3  04015-3.csv           N
4  04015-4.csv           N


100%|████████████████████████████████████| 2448/2448 [00:00<00:00, 12061.55it/s]


      subsetID rhythmLabel
0  04043-0.csv           N
1  04043-1.csv           N
2  04043-2.csv           N
3  04043-3.csv           N
4  04043-4.csv           N


100%|████████████████████████████████████| 1562/1562 [00:00<00:00, 19691.15it/s]


...

      subsetID rhythmLabel
0  08405-0.csv           N
1  08405-1.csv           N
2  08405-2.csv           N
3  08405-3.csv           N
4  08405-4.csv           N


100%|████████████████████████████████████| 1593/1593 [00:00<00:00, 43388.22it/s]


      subsetID rhythmLabel
0  08434-0.csv           N
1  08434-1.csv           N
2  08434-2.csv           N
3  08434-3.csv           N
4  08434-4.csv           N


100%|█████████████████████████████████████| 2379/2379 [00:00<00:00, 9505.13it/s]


      subsetID rhythmLabel
0  08455-0.csv           N
1  08455-1.csv           N
2  08455-2.csv           N
3  08455-3.csv           N
4  08455-4.csv           N
